In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import plotly.express as px
import psycopg2
from folium.plugins import HeatMap
import rtree
%matplotlib inline

In [ ]:
pip install plotly

In [ ]:
zipcode_population_clean = pd.read_csv('zipcode_population_clean.csv')


In [ ]:
zipcodes_geojson_data = gpd.read_file('Zip Codes.geojson')


In [ ]:
zipcode_population_clean['ZipCode'] = zipcode_population_clean['ZipCode'].astype(str)


In [ ]:
zipcodes_geojson_data = gpd.read_file('Zip Codes.geojson')

In [ ]:
print(zipcodes_geojson_data.columns)

In [ ]:
zipcodes_with_population = zipcodes_geojson_data.merge(zipcode_population_clean, how="left", left_on="zipcode", right_on="ZipCode")


In [ ]:
zipcode_population_clean['ZipCode'] = zipcode_population_clean['ZipCode'].astype(str)
zipcodes_geojson_data['zipcode'] = zipcodes_geojson_data['zipcode'].astype(str)

In [ ]:
zipcodes_with_population = zipcodes_geojson_data.merge(zipcode_population_clean, how="left", left_on="zipcode", right_on="ZipCode")


In [ ]:
fig = px.choropleth_mapbox(zipcodes_with_population,
                           geojson=zipcodes_with_population.geometry,
                           locations=zipcodes_with_population.index,
                           color='Population',
                           hover_name='poname', 
                           hover_data={'ZipCode': True, 'Population': True},
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           zoom=9,
                           center={"lat": 36.1627, "lon": -86.7816},
                           opacity=0.5,
                           labels={'Population': 'Population'},
                           title='Nashville Zip Code Areas by Population')

In [ ]:
fig.update_layout(coloraxis_colorbar=dict(title="Population"))
fig.show()

In [ ]:
fig.update_layout(height=700)

In [ ]:
html_file_path = '../data/nashville_zipcode_map.html'
fig.write_html(html_file_path, include_plotlyjs='cdn')

In [ ]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt

db_name = 'scooter'
db_user = 'postgres'
db_password = 'postgres'
db_host = 'localhost'
db_port = '5432'

colors = {
    'May': 'skyblue',
    'June': 'lightgreen',
    'July': 'salmon'
}


conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_password, host=db_host, port=db_port)

# Months of interest
months = {
    'May': 5,
    'June': 6,
    'July': 7
}

for month_name, month_number in months.items():
    sql_query = f"""
    SELECT day_of_week, COUNT(*) AS trip_count
    FROM (
        SELECT to_char(startdate, 'Dy') AS day_of_week
        FROM trips
        WHERE EXTRACT(MONTH FROM startdate) = {month_number}
    ) AS subquery
    GROUP BY day_of_week
    ORDER BY CASE
             WHEN day_of_week = 'Sun' THEN 1
             WHEN day_of_week = 'Mon' THEN 2
             WHEN day_of_week = 'Tue' THEN 3
             WHEN day_of_week = 'Wed' THEN 4
             WHEN day_of_week = 'Thu' THEN 5
             WHEN day_of_week = 'Fri' THEN 6
             WHEN day_of_week = 'Sat' THEN 7
             END;
    """

    df = pd.read_sql_query(sql_query, conn)

    plt.figure(figsize=(10, 6))
    plt.bar(df['day_of_week'], df['trip_count'], color=colors[month_name])
    plt.title(f'Trip Count Distribution by Day of Week for {month_name}')
    plt.xlabel('Day of Week')
    plt.ylabel('Trip Count')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

conn.close()


In [ ]:
heat_csv_path = '../data/heat.csv'
heat_data = pd.read_csv(heat_csv_path)

In [ ]:
heat_data_head = heat_data.head()

heat_data_head

In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize=(10, 6))


plt.scatter(heat_data['startlongitude'], heat_data['startlatitude'], 
            alpha=0.5, c=heat_data['trip_count'], cmap='Reds', 
            s=heat_data['trip_count'] * 10)


plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Heatmap of Scooter Usage in Nashville for May 2019')


plt.colorbar(label='Number of Trips')

# Show the plot
plt.show()


In [ ]:
def add_custom_legend(folium_map, title, colors, labels):
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    legend_html = f'''
     <div style="position: fixed; 
                 bottom: 200px; left: 50px; width: 100px; height: 100px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 ">&nbsp; <b>{title}</b> <br>
     '''
    for color, label in zip(colors, labels):
        legend_html += f'<i style="background:{color};width:10px;height:10px;display:inline-block;"></i>{label}<br>'
    legend_html += '</div>'

    folium_map.get_root().html.add_child(folium.Element(legend_html))

In [ ]:
heat_csv_path = '../data/heat.csv'
heat_data = pd.read_csv(heat_csv_path)

In [ ]:
heat_data_points = [[row['startlatitude'], row['startlongitude'], row['trip_count']] 
                    for index, row in heat_data.iterrows()]

In [ ]:
nashville_center = [36.1627, -86.7816]
map_nashville = folium.Map(location=nashville_center, tiles='OpenStreetMap', zoom_start=13)


In [ ]:
heatmap_layer = HeatMap(heat_data_points, radius=10, max_zoom=1, 
                        gradient={0.2: 'green', 0.4: 'lime', 0.6: 'yellow', 0.8: 'orange', 1: 'red'}, 
                        blur=13)
map_nashville.add_child(heatmap_layer)

In [ ]:
legend_title = "Trip Count"
legend_colors = ['green', 'lime', 'yellow', 'orange', 'red']
legend_labels = ['Very Low', 'Low', 'Moderate', 'High', 'Very High']
add_custom_legend(map_nashville, legend_title, legend_colors, legend_labels)

In [ ]:
heatmap_html_path = '../data/scooter_heatmap_with_custom_legend.html'
map_nashville.save(heatmap_html_path)

In [3]:
trip_coordinates = pd.read_csv('../data/tripcoordinates.csv')

In [5]:
m = folium.Map(location=[trip_coordinates['startlatitude'].mean(), 
                         trip_coordinates['startlongitude'].mean()], 
               zoom_start=13)

In [6]:
marker_cluster = MarkerCluster().add_to(m)

In [10]:
for idx, row in trip_coordinates.iterrows():

    start_location = [row['startlatitude'], row['startlongitude']]
    end_location = [row['endlatitude'], row['endlongitude']]

    folium.CircleMarker(start_location, radius=1, color='green').add_to(marker_cluster)
    folium.CircleMarker(end_location, radius=1, color='red').add_to(marker_cluster)

    folium.PolyLine([start_location, end_location], color='blue', weight=1.5, opacity=0.2).add_to(m)


In [11]:
m